<a href="https://colab.research.google.com/github/anggarisaputra23/assignmentop/blob/main/Genetic_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:


# Assume the following are defined outside the function
# production_line_tasks: a list of dictionaries, each representing a task
#   e.g., [{'name': 'TaskA', 'duration': 5, 'dependencies': []}, ...]
#   'dependencies' is a list of names of tasks that must complete before this task starts.
# machines: a list of available machines, e.g., ['Machine1', 'Machine2']
# machine_speeds: a dictionary mapping machine names to their processing speed relative to a base, e.g., {'Machine1': 1.0, 'Machine2': 1.2}

import random
import copy
import itertools

# --- Fitness Evaluation for Scheduling ---
def evaluate_scheduling_fitness(schedule, production_line_tasks, machine_speeds):
  """
  Evaluates the fitness of a scheduling chromosome.
  Fitness is the negative of the total completion time (makespan),
  as we want to minimize makespan. Higher fitness is better.
  A valid schedule must respect task dependencies.
  """
  # Decode the schedule chromosome into a sequence of task assignments to machines
  # The chromosome is a permutation of tasks. We need to assign them to machines.
  # A simple decoding could be assigning tasks sequentially to available machines.
  # This is a simplified decoding; more complex decodings exist.

  # Let's assume the chromosome is a permutation of task indices.
  # We will assign tasks in this order to the machine that becomes free earliest.

  num_tasks = len(production_line_tasks)
  num_machines = len(machine_speeds)
  machine_finish_times = {machine: 0 for machine in machine_speeds}
  task_completion_times = {}

  # Create a dictionary for task objects for easy lookup
  task_dict = {task['name']: task for task in production_line_tasks}

  # Create a dictionary to track dependencies completion
  task_dependencies_met = {task['name']: False for task in production_line_tasks}
  # Mark tasks with no dependencies as met initially
  for task in production_line_tasks:
      if not task.get('dependencies'):
          task_dependencies_met[task['name']] = True

  # Convert chromosome (task index permutation) to task name permutation
  task_order = [production_line_tasks[i]['name'] for i in schedule]

  # Process tasks in the order given by the schedule
  for task_name in task_order:
      task = task_dict[task_name]
      dependencies_met = all(task_completion_times.get(dep, 0) > 0 for dep in task.get('dependencies', []))
      if not dependencies_met:
          # This schedule is invalid as dependencies are not met in order.
          # Return a very low fitness value.
          return -float('inf')

      # Find the machine that becomes free earliest
      earliest_machine = min(machine_finish_times, key=machine_finish_times.get)
      earliest_free_time = machine_finish_times[earliest_machine]

      # Calculate task start time on this machine
      dependency_completion_time = 0
      for dep_name in task.get('dependencies', []):
          dependency_completion_time = max(dependency_completion_time, task_completion_times.get(dep_name, 0))

      task_start_time = max(earliest_free_time, dependency_completion_time)

      # Calculate task duration on this machine considering machine speed
      task_duration = task['duration'] / machine_speeds[earliest_machine]

      # Calculate task completion time
      task_completion_time = task_start_time + task_duration

      # Update machine finish time and task completion time
      machine_finish_times[earliest_machine] = task_completion_time
      task_completion_times[task_name] = task_completion_time

  # The total completion time (makespan) is the maximum of all machine finish times
  makespan = max(machine_finish_times.values())

  # Fitness is the negative of the makespan (since we minimize makespan)
  return -makespan

# --- Chromosome Representation for Scheduling ---
# A chromosome is a permutation of task indices.
# The length of the chromosome is the number of tasks.
def create_scheduling_chromosome(num_tasks):
  """Creates a random permutation of task indices."""
  chromosome = list(range(num_tasks))
  random.shuffle(chromosome)
  return chromosome

# --- Selection ---
# Reuse the previous select_parent function. It works with any fitness scores.

# --- Crossover for Scheduling (Order Crossover - OX) ---
# Since the chromosome is a permutation, simple one-point crossover can break the permutation property.
# Order Crossover (OX) preserves the permutation.
def order_crossover(parent1, parent2):
  """Performs Order Crossover (OX) on two parent chromosomes (permutations)."""
  size = len(parent1)
  a, b = random.sample(range(size), 2)
  if a > b:
    a, b = b, a

  # Copy a segment from parent1 to child1
  child1 = [None] * size
  child1[a:b+1] = parent1[a:b+1]

  # Fill the remaining slots in child1 with elements from parent2
  # in the order they appear after point b, wrapping around
  p2_index = (b + 1) % size
  c1_index = (b + 1) % size
  while None in child1:
      if parent2[p2_index] not in child1:
          child1[c1_index] = parent2[p2_index]
          c1_index = (c1_index + 1) % size
      p2_index = (p2_index + 1) % size

  # Repeat for child2 with parent1 and parent2 swapped
  child2 = [None] * size
  child2[a:b+1] = parent2[a:b+1]

  p1_index = (b + 1) % size
  c2_index = (b + 1) % size
  while None in child2:
      if parent1[p1_index] not in child2:
          child2[c2_index] = parent1[p1_index]
          c2_index = (c2_index + 1) % size
      p1_index = (p1_index + 1) % size

  return child1, child2


# --- Mutation for Scheduling (Swap Mutation) ---
# Swap mutation preserves the permutation.
def swap_mutation(chromosome, mutation_rate):
  """Mutates a permutation chromosome by swapping two elements with a given probability."""
  mutated_chromosome = chromosome[:]
  if random.random() < mutation_rate:
    idx1, idx2 = random.sample(range(len(mutated_chromosome)), 2)
    mutated_chromosome[idx1], mutated_chromosome[idx2] = mutated_chromosome[idx2], mutated_chromosome[idx1]
  return mutated_chromosome

# --- Genetic Algorithm for Scheduling ---
def genetic_algorithm_scheduling(production_line_tasks, machine_speeds, population_size, generations, mutation_rate):
  num_tasks = len(production_line_tasks)
  if num_tasks == 0:
      return [] # No tasks to schedule

  # Initialize population
  population = [create_scheduling_chromosome(num_tasks) for _ in range(population_size)]

  best_overall_chromosome = None
  best_overall_fitness = -float('inf')

  for generation in range(generations):
    # Evaluate fitness for the current population
    fitness_scores = [evaluate_scheduling_fitness(chromosome, production_line_tasks, machine_speeds) for chromosome in population]

    # Find the best chromosome in the current generation
    current_best_fitness = max(fitness_scores)
    current_best_index = fitness_scores.index(current_best_fitness)
    current_best_chromosome = population[current_best_index]

    # Update overall best if the current generation's best is better
    if current_best_fitness > best_overall_fitness:
        best_overall_fitness = current_best_fitness
        best_overall_chromosome = copy.deepcopy(current_best_chromosome)

    # Print best fitness of the generation
    print(f"Generation {generation}: Best Fitness = {current_best_fitness} (Makespan = {-current_best_fitness})")

    # Selection (using the existing select_parent)
    parents = []
    # Handle cases where all fitness scores are -inf (invalid schedules)
    valid_indices = [i for i, fitness in enumerate(fitness_scores) if fitness > -float('inf')]
    if not valid_indices:
        print("Warning: All schedules in this generation are invalid. Re-initializing population.")
        population = [create_scheduling_chromosome(num_tasks) for _ in range(population_size)]
        continue # Skip crossover and mutation for this generation

    valid_population = [population[i] for i in valid_indices]
    valid_fitness_scores = [fitness_scores[i] for i in valid_indices]

    if len(valid_population) < 2:
         # If less than 2 valid parents, just duplicate the valid ones or re-initialize
         print("Warning: Less than 2 valid parents available. Re-initializing population.")
         population = [create_scheduling_chromosome(num_tasks) for _ in range(population_size)]
         continue

    for _ in range(population_size):
      parents.append(select_parent(valid_population, valid_fitness_scores))

    # Create next generation through crossover and mutation
    next_population = []
    # Ensure an even number of parents for crossover
    if len(parents) % 2 != 0:
        parents.append(random.choice(parents)) # Add a random parent to make it even

    for i in range(0, len(parents), 2):
      parent1 = parents[i]
      parent2 = parents[i+1]
      child1, child2 = order_crossover(parent1, parent2)
      next_population.append(swap_mutation(child1, mutation_rate))
      next_population.append(swap_mutation(child2, mutation_rate))

    population = next_population[:population_size] # Ensure population size is maintained

  # Return the best chromosome found across all generations
  return best_overall_chromosome

# Example Usage:

# Define your production line tasks
# Duration is a base duration, actual duration depends on the assigned machine speed
production_line_tasks = [
    {'name': 'TaskA', 'duration': 10, 'dependencies': []},
    {'name': 'TaskB', 'duration': 8, 'dependencies': ['TaskA']},
    {'name': 'TaskC', 'duration': 12, 'dependencies': ['TaskA']},
    {'name': 'TaskD', 'duration': 7, 'dependencies': ['TaskB', 'TaskC']},
    {'name': 'TaskE', 'duration': 9, 'dependencies': ['TaskD']},
]

# Define your machines and their speeds
machines = ['Machine1', 'Machine2']
machine_speeds = {'Machine1': 1.0, 'Machine2': 1.5} # Machine2 is 1.5 times faster

# Genetic Algorithm Parameters
population_size = 100
generations = 500
mutation_rate = 0.1

# Run the genetic algorithm
best_schedule_chromosome = genetic_algorithm_scheduling(
    production_line_tasks,
    machine_speeds,
    population_size,
    generations,
    mutation_rate
)

print("\nBest Schedule Chromosome (Task Indices Permutation):", best_schedule_chromosome)

# To interpret the best schedule, you would decode the chromosome:
# Example decoding to show task order:
if best_schedule_chromosome is not None:
    best_task_order = [production_line_tasks[i]['name'] for i in best_schedule_chromosome]
    print("Best Task Order:", best_task_order)
    # You could further simulate this schedule to get the actual start/finish times on each machine

    # Calculate the fitness of the best chromosome found
    best_makespan_fitness = evaluate_scheduling_fitness(
        best_schedule_chromosome,
        production_line_tasks,
        machine_speeds
    )
    print(f"Achieved Makespan: {-best_makespan_fitness}")


Generation 0: Best Fitness = -31.0 (Makespan = 31.0)
Generation 1: Best Fitness = -31.0 (Makespan = 31.0)
Generation 2: Best Fitness = -35.66666666666667 (Makespan = 35.66666666666667)
Generation 3: Best Fitness = -35.66666666666667 (Makespan = 35.66666666666667)
Generation 4: Best Fitness = -31.0 (Makespan = 31.0)
Generation 5: Best Fitness = -31.0 (Makespan = 31.0)
Generation 6: Best Fitness = -35.66666666666667 (Makespan = 35.66666666666667)
Generation 7: Best Fitness = -31.0 (Makespan = 31.0)
Generation 8: Best Fitness = -31.0 (Makespan = 31.0)
Generation 9: Best Fitness = -31.0 (Makespan = 31.0)
Generation 10: Best Fitness = -31.0 (Makespan = 31.0)
Generation 11: Best Fitness = -31.0 (Makespan = 31.0)
Generation 12: Best Fitness = -31.0 (Makespan = 31.0)
Generation 13: Best Fitness = -31.0 (Makespan = 31.0)
Generation 14: Best Fitness = -35.66666666666667 (Makespan = 35.66666666666667)
Generation 15: Best Fitness = -31.0 (Makespan = 31.0)
Generation 16: Best Fitness = -31.0 (Makes